# Code for interacting with API of openFDA

## Plan
In this project, we are looking for device adverse events. We are particularly interested in:
    - Cause of failure
    - Date of failure (ie age of device)
We want to see which is the most common cause for given device to fail and at which stage of use this occurs.
Relevant fields could be (for full reference see https://open.fda.gov/device/event/reference/):
### Event
device_date_of_manufacturer
date_of_event, date_report, date_received
previous_use_code, remedial_action
single_use_flag
### Source
reprocessed_and_reused_flag
### Device
device.generic_name
device.expiration_date_of_device, device.device_age_text
device.implant_flag, device.date_removed_flag
device.manufacturer_d_name, device.manufacturer_d_state, device.manufacturer_d_country
### Patient
patient.sequence_number_outcome, patient.sequence_number_treatment
### Report Text
mdr_text.text, mdr_text.text_type_code
### Reporter Dependent Fields
#### By user facility / importer
report_date
event_location
manufacturer_name, manufacturer_country
manufacturer_g1_name, manufacturer_g1_state
### OpenFDA fields
device_class
### Further interesting fields:
Source: reporter_occupation_code
Device: device.device_operator

In [1]:
import numpy as np
import pandas as pd
import json
import requests

baseurl = 'https://api.fda.gov/device/event.json?'
apikey = ''
with open('apikey.txt', 'r') as myfile:
    apikey = myfile.read().replace('\n', '')

In [105]:
# Example of quering, for complete guide go to: https://open.fda.gov/api/
# query = 'search=(device.generic_name:"stent"+AND+reprocessed_and_reused_flag:"Y"+AND+date_of_event:' + \
# '["20150324"+TO+"20170324"]+AND+(_exists_:device_date_of_manufacturer+AND+_exists_:date_of_event))&limit=10'
# # query2 = '_exists_:search=()'
# q1 = baseurl + 'api_key=' + apikey + '&' + query

In [41]:
# Example of quering, for complete guide go to: https://open.fda.gov/api/
start_date = '20150324'
end_date = '20170324'
query = 'search='
limit = 20

In [42]:
list_features = ['device_date_of_manufacturer', # features to check for existance
                 'date_of_event',
                 'date_report',
                 'date_received',
                 'previous_use_code',
                 'remedial_action',
                 'single_use_flag',
                 'reprocessed_and_reused_flag',
                 #'reporter_occupation_code',
                 'device.date_received',
                 'device.generic_name' # this allows for empty string! 
                ]

list_features2 = [#'device.generic_name="pump"',
                  #'device.implant_flag:"Y"',
                  'previous_use_code:"I"', # I - initial use, R - reuse, U - unknown, * - invalid data
                  'device.manufacturer_d_country:"US"' # SZ - Switzerland
                 ]
                  
    
# adding date range
query = query+"date_of_event:[\""+start_date+"\""+"+TO+"+"\""+end_date+"\"]"
for x in list_features:
    query = query + "+AND+_exists_:" + x
for y in list_features2:
    query = query + "+AND+" + y

q1 = baseurl + 'api_key=' + apikey + '&' + query + '&' + 'limit=' + str(limit)
q1

'https://api.fda.gov/device/event.json?api_key=znAk3nEbFHqx6BMuiw0WRN5Qmg2WuSNelj2wkWnE&search=date_of_event:["20150324"+TO+"20170324"]+AND+_exists_:device_date_of_manufacturer+AND+_exists_:date_of_event+AND+_exists_:date_report+AND+_exists_:date_received+AND+_exists_:previous_use_code+AND+_exists_:remedial_action+AND+_exists_:single_use_flag+AND+_exists_:reprocessed_and_reused_flag+AND+_exists_:device.date_received+AND+_exists_:device.generic_name+AND+previous_use_code:"I"+AND+device.manufacturer_d_country:"US"&limit=20'

In [43]:
dq1 = requests.get(q1)
# dq1.json()['results']
data = json.loads(dq1.text)
number = data['meta']['results']['total'] # check number of matching entries
results = data['results']
number

334176

In [5]:
# Can also spare some structuring effort when loading data by calling normalization method
# dftest = pd.io.json.json_normalize(results)

In [44]:
results[3]

{'adverse_event_flag': 'Y',
 'date_manufacturer_received': '20160419',
 'date_of_event': '20150609',
 'date_received': '20120510',
 'date_report': '20121119',
 'device': [{'brand_name': 'ARTICULEZE M HEAD 36MM +5',
   'catalog_number': '136552000',
   'date_received': '20120510',
   'date_removed_flag': '',
   'device_age_text': 'DA',
   'device_availability': 'No',
   'device_evaluated_by_manufacturer': 'R',
   'device_event_key': '',
   'device_operator': 'HEALTH PROFESSIONAL',
   'device_report_product_code': 'JDI',
   'device_sequence_number': ' 1.0',
   'generic_name': 'FEMORAL HEAD',
   'implant_flag': '',
   'lot_number': '2168606',
   'manufacturer_d_address_1': 'ST. ANTHONYS RD',
   'manufacturer_d_address_2': '',
   'manufacturer_d_city': 'LEEDS',
   'manufacturer_d_country': 'US',
   'manufacturer_d_name': 'DEPUY INTERNATIONAL',
   'manufacturer_d_postal_code': '',
   'manufacturer_d_state': '',
   'manufacturer_d_zip_code': '',
   'manufacturer_d_zip_code_ext': '',
   'mode

In [45]:
# Fields of Interest
fois_result = ['device_date_of_manufacturer', 'date_of_event']
fois_device = ['generic_name', 'expiration_date_of_device', 'device_age_text', 'implant_flag', 'date_removed_flag', \
               'manufacturer_d_name', 'manufacturer_d_state', 'manufacturer_d_country']
fois_patient = ['sequence_number_outcome', 'sequence_number_treatment']
fois_mdrText = ['text', 'text_type_code']
fois_openfda = ['device_name', 'device_class', 'medical_specialty_description']

# device = data['results'][0]['device'][0]
device = [x['device'][0] for x in data['results']]
# patient = data['results'][0]['patient'][0]
patient = [x['patient'][0] for x in data['results']]
# mdrText = data['results'][0]['mdr_text'][0] # there may be more items in the list! 
mdrText = [x['mdr_text'] for x in data['results']]
#mdrText = [y['text'] for y in [x['mdr_text'][0] for x in data['results']]]
# openfda = data['results'][0]['device'][0]['openfda']
openfda = [x['device'][0]['openfda'] for x in data['results']]

In [46]:
fillDic = {'mdr_text_key': '', 'patient_sequence_number': '', 'text': np.nan, 'text_type_code': np.nan}
a = [x[0] if len(x) > 0 else fillDic for x in mdrText]
b = [x[1] if len(x) > 1 else fillDic for x in mdrText] # some of them have even three entries....

In [47]:
df_results = pd.DataFrame(results, index = range(len(results)), columns = fois_result)
df_openfda = pd.DataFrame(openfda, index = range(len(results)),columns = fois_openfda)
df_device = pd.DataFrame(device, index = range(len(results)),columns = fois_device)
df_patient = pd.DataFrame(patient, index = range(len(results)),columns = fois_patient)
# df_mdrText = pd.DataFrame(mdrText, index = range(len(results)),columns = fois_mdrText)

# df = pd.concat([df_device, df_patient, df_mdrText, df_openfda], axis = 1)

In [49]:
a = pd.DataFrame(a, index = range(len(results)),columns = fois_mdrText)
b = pd.DataFrame(b, index = range(len(results)),columns = fois_mdrText)
df_mdrText = pd.concat([a, b], axis = 1)
df = pd.concat([df_results, df_device, df_patient, df_mdrText, df_openfda], axis = 1)

In [50]:
df

,device_date_of_manufacturer,date_of_event,generic_name,expiration_date_of_device,device_age_text,implant_flag,date_removed_flag,manufacturer_d_name,manufacturer_d_state,manufacturer_d_country,sequence_number_outcome,sequence_number_treatment,text,text_type_code,text,text_type_code,device_name,device_class,medical_specialty_description
0,20071101,20160715,PULMONARY VALVE PROSTHESIS PERCUTANEOUSLY DELI...,20081101,NA,,,"MEDTRONIC HEART VALVES, INC.",CA,US,[8.],[],(B)(4). EVAL METHOD: DEVICE HISTORY REVIEWED. ...,Additional Manufacturer Narrative,MEDTRONIC RECEIVED INFO THAT THE PT IMPLANTED ...,Description of Event or Problem,Pulmonary Valve Prosthesis Percutaneously Deli...,f,Unknown
1,20090701,20151201,ZIMMER MMC CUP,NaN,DA,,,"ZIMMER, INC.",IN,US,[8.],[],THE MANUFACTURER DID NOT RECEIVE EXPLANTED DEV...,Additional Manufacturer Narrative,IT HAS NOW BEEN REPORTED THAT THE PATIENT WAS ...,Description of Event or Problem,"Prosthesis, Hip, Semi-Constrained, Metal/Polym...",2,Orthopedic
2,20060710,20150609,HIP LINER,NaN,DA,,,DEPUY INTERNATIONAL,,US,[8.],[],THE DEVICE ASSOCIATED WITH THIS REPORT WAS NOT...,Additional Manufacturer Narrative,ADDITIONAL NARRATIVE: NO DEVICE ASSOCIATED WIT...,Additional Manufacturer Narrative,"Prosthesis, Hip, Semi-Constrained (Metal Uncem...",3,Orthopedic
3,20060608,20150609,FEMORAL HEAD,NaN,DA,,,DEPUY INTERNATIONAL,,US,[8.],[],THE DEVICE ASSOCIATED WITH THIS REPORT WAS NOT...,Additional Manufacturer Narrative,NO DEVICE ASSOCIATED WITH THIS REPORT WAS RECE...,Additional Manufacturer Narrative,"Prosthesis, Hip, Semi-Constrained, Metal/Polym...",2,Orthopedic
4,20061201,20150423,"HIP COMPONENT, CODE:KWA",20141231,DA,,,"WRIGHT MEDICAL TECHNOLOGY, INC.",TN,US,[8.],[],(B)(4). THIS IS THE SAME EVENT AS 3010536692-2...,Additional Manufacturer Narrative,ALLEGEDLY PATIENT WAS REVISED DUE TO MOM COMPL...,Description of Event or Problem,"Prosthesis, Hip, Semi-Constrained (Metal Uncem...",3,Orthopedic
5,20050808,20150406,FEMORAL HEAD,20100808,DA,,,DEPUY INTERNATIONAL,,US,"[1. O, 2. R]",[],UPDATE: (B)(4) 2012 - PFS WAS RECEIVED FROM LE...,Description of Event or Problem,THIS COMPLAINT IS STILL UNDER INVESTIGATION. D...,Additional Manufacturer Narrative,"Prosthesis, Hip, Semi-Constrained, Metal/Polym...",2,Orthopedic
6,20060114,20150406,HIP LINER,20110114,DA,,,DEPUY INTERNATIONAL,,US,"[1. O, 2. R]",[],THE DEVICE ASSOCIATED WITH THIS REPORT WAS NOT...,Additional Manufacturer Narrative,LITIGATION ALLEGES THAT THE PATIENT SUFFERS FR...,Description of Event or Problem,"Prosthesis, Hip, Semi-Constrained (Metal Uncem...",3,Orthopedic
7,20050628,20150520,"SYSTEM, ENDOVASCULAR GRAFT, AORTIC ANEURYSM TR...",20070629,DA,,,MEDTRONIC CARDIOVASCULAR SANTA ROSA,CA,US,[1. R],[],"(B)(4). EVALUATION CODES, RESULTS: INHERENT RI...",Additional Manufacturer Narrative,ADDITIONAL INFORMATION WAS RECEIVED FOR THIS C...,Description of Event or Problem,"System, Endovascular Graft, Aortic Aneurysm Tr...",3,Unknown
8,20041006,20150615,"PROSTHESIS, HIP",20141031,DA,,,BIOMET ORTHOPEDICS,IN,US,"[1. H, 2. R]",[],EVENT IS BEING REPORTED TO FDA ON ONE MEDWATCH...,Additional Manufacturer Narrative,THIS FOLLOW-UP REPORT IS BEING FILED TO RELAY ...,Additional Manufacturer Narrative,"Prosthesis, Hip, Semi-Constrained (Metal Uncem...",3,Orthopedic
9,20040112,20160322,"PROSTHESIS, HIP",20140131,DA,,,BIOMET ORTHOPEDICS,IN,US,"[8. , 8.]",[],LEGAL COUNSEL FOR PATIENT REPORTED THAT THE PA...,Description of Event or Problem,LEGAL COUNSEL FOR PATIENT REPORTED THAT THE PA...,Description of Event or Problem,"Prosthesis, Hip, Hemi-, Femoral, Metal/Polymer...",2,Orthopedic
